In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

In [2]:
def calculate_statistics(group):
    r2_test = group['r2_test']
    r2_test_dict = {f'run{i}': r2_test_val for i, r2_test_val in enumerate(r2_test)}
    return pd.Series({
        **r2_test_dict, 
        'r2_test_mean': np.mean(r2_test),
        'r2_test_max': np.max(r2_test),
        'r2_test_min': np.min(r2_test),
        'r2_test_std': np.std(r2_test, ddof=0),
    })

def calculate_statistics2(group):
    rmse_test = group['rmse_test']
    rmse_test_dict = {f'run{i}': rmse_test_val for i, rmse_test_val in enumerate(rmse_test)}
    return pd.Series({
        **rmse_test_dict, 
        'rmse_test_mean': np.mean(rmse_test),
        'rmse_test_max': np.max(rmse_test),
        'rmse_test_min': np.min(rmse_test),
        'rmse_test_std': np.std(rmse_test, ddof=0),
    })

In [3]:
results_r2 = []
results_rmse = []
for i in range(10):
    for dataset in ['RDKit', 'mordred']:
        data = pd.read_csv(f'data/data_{dataset}.csv')
        y = pd.DataFrame(data['Yield'],columns=['Yield'])
        X = data.drop(columns=['Name', 'ID', 'Yield'])
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=i)
        param = {"n_estimators": [100, 1000, 5000], "max_depth": [3, 4, 5, 6]}
        reg = GridSearchCV(RandomForestRegressor(random_state=0), param_grid=param, cv=10, n_jobs=6)
        reg.fit(X_train,y_train['Yield'])
        best = reg.best_estimator_
        y_pred1 = best.predict(X_train)
        y_pred2 = best.predict(X_test)
        r2_test_score = metrics.r2_score(y_test, y_pred2)
        rmse_test_score = metrics.root_mean_squared_error(y_test, y_pred2)
        results_r2.append({'desc': dataset,'r2_test': r2_test_score})
        results_rmse.append({'desc': dataset, 'rmse_test': rmse_test_score})

results_df = pd.DataFrame(results_r2)
gen_results = results_df.groupby(['desc']).apply(calculate_statistics).reset_index()
results_df2 = pd.DataFrame(results_rmse)
gen_results2 = results_df2.groupby(['desc']).apply(calculate_statistics2).reset_index()

In [4]:
gen_results.T.to_csv('result/result_r2.csv', header=False)
gen_results.T

,0,1
desc,RDKit,mordred
run0,0.639458,0.525053
run1,0.743784,0.40602
run2,0.661759,0.522364
run3,0.747078,0.743019
run4,0.653114,0.561296
run5,0.618851,0.39854
run6,0.728731,0.675283
run7,0.726342,0.57009
run8,0.713035,0.633687


In [5]:
gen_results2.T.to_csv('result/result_rmse.csv', header=False)
gen_results2.T

,0,1
desc,RDKit,mordred
run0,21.74867,24.961906
run1,16.660187,25.366631
run2,22.499459,26.736661
run3,17.741714,17.883537
run4,21.863687,24.587584
run5,21.400811,26.883528
run6,18.043526,19.741218
run7,19.302813,24.193892
run8,19.179418,21.669402
